# Digital Phenomenology Lab - Visualization Tutorial

This notebook demonstrates how to use the visualization tools to analyze experiment data.

In [ ]:
# Setup
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Import visualization tools
from src.db.experiment_database import ExperimentDatabase
from src.analysis.visualizations import (
    TimelinePlot,
    BeliefEvolutionPlot,
    MemoryCorruptionPlot,
    MultiExperimentComparison
)

# Enable inline plotting
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Connect to Experiment Database

In [ ]:
# Connect to database
db = ExperimentDatabase('logs/experiments.db')

# List all experiments
experiments = db.list_experiments()
print(f"Found {len(experiments)} experiments")

for exp in experiments:
    print(f"  - {exp['experiment_id']}: {exp['name']} ({exp['mode']})")

## 2. Timeline Visualization

Visualize experiment events chronologically.

In [ ]:
# Select an experiment
experiment_id = experiments[0]['experiment_id'] if experiments else 'demo_exp'

# Get experiment data
exp_data = db.get_experiment(experiment_id)

if exp_data:
    # Gather events from different tables
    events = []
    
    # Get self-reports
    reports = db.get_self_reports(experiment_id)
    for report in reports:
        events.append({
            'type': 'self_report',
            'timestamp': report['timestamp'],
            'description': f"Q: {report['question'][:30]}..."
        })
    
    # Get interventions
    interventions = db.get_interventions(experiment_id)
    for intervention in interventions:
        events.append({
            'type': 'intervention',
            'timestamp': intervention['timestamp'],
            'description': intervention['description'],
            'annotate': True
        })
    
    # Create timeline plot
    timeline = TimelinePlot(exp_data)
    fig = timeline.plot_static(events)
else:
    print("No experiment data found. Run an experiment first!")

## 3. Belief Evolution

Track how beliefs change across cycles.

In [ ]:
# Get belief evolution data
if exp_data:
    # Query epistemic assessments
    belief_types = ['memory_continuity', 'self_identity', 'reality_perception']
    belief_data = {}
    
    for belief_type in belief_types:
        assessments = db.get_belief_evolution(experiment_id, belief_type)
        if assessments:
            belief_data[belief_type] = assessments
    
    if belief_data:
        plotter = BeliefEvolutionPlot(experiment_id)
        fig = plotter.plot_belief_evolution(belief_data)
    else:
        print("No belief data available")
else:
    print("No experiment selected")

## 4. Memory Corruption Analysis

Visualize memory corruption patterns.

In [ ]:
if exp_data:
    # Get messages with corruption flags
    messages = db.get_messages(experiment_id)
    
    if messages:
        plotter = MemoryCorruptionPlot(experiment_id)
        fig = plotter.plot_corruption_heatmap(messages)
    else:
        print("No message data available")
else:
    print("No experiment selected")

## 5. Multi-Experiment Comparison

Compare metrics across different experiments.

In [ ]:
if len(experiments) > 1:
    # Get full data for all experiments
    exp_data_list = [db.get_experiment(exp['experiment_id']) for exp in experiments]
    exp_data_list = [exp for exp in exp_data_list if exp]  # Filter None values
    
    if exp_data_list:
        comparison = MultiExperimentComparison(exp_data_list)
        
        # Dashboard
        fig = comparison.plot_summary_dashboard()
else:
    print("Need at least 2 experiments for comparison")

## 6. Network Visualizations (Multi-Agent)

For HIVE and SPLIT_BRAIN experiments.

In [ ]:
try:
    from src.analysis.network_graphs import (
        CommunicationNetwork,
        BeliefAlignmentNetwork,
        InfluenceGraph
    )
    
    # Find a multi-agent experiment
    multi_agent_exp = None
    for exp in experiments:
        if exp['mode'] in ['HIVE', 'SPLIT_BRAIN']:
            multi_agent_exp = exp
            break
    
    if multi_agent_exp:
        exp_id = multi_agent_exp['experiment_id']
        messages = db.get_messages(exp_id)
        
        # Communication network
        comm_net = CommunicationNetwork(exp_id, multi_agent_exp['mode'])
        fig = comm_net.plot_communication_network(messages)
    else:
        print("No multi-agent experiments found")
        
except ImportError:
    print("NetworkX not installed. Install with: pip install networkx")

## 7. Interactive Visualizations with Plotly

Create interactive plots that you can zoom and explore.

In [ ]:
try:
    import plotly.graph_objects as go
    
    if exp_data and events:
        timeline = TimelinePlot(exp_data)
        fig = timeline.plot_interactive(events)
        fig.show()
    else:
        print("No data available for interactive plot")
        
except ImportError:
    print("Plotly not installed. Install with: pip install plotly")

## 8. Export Visualizations

Save plots in multiple formats for papers and presentations.

In [ ]:
from src.analysis.visualizations import save_plot_multiple_formats

# Create a plot
if exp_data and events:
    timeline = TimelinePlot(exp_data)
    fig = timeline.plot_static(events, show=False)
    
    # Save in multiple formats
    save_plot_multiple_formats(
        fig, 
        'exports/timeline',
        formats=['png', 'pdf', 'svg']
    )
    
    print("Plots saved!")
else:
    print("No data to export")

## Summary

This notebook demonstrated:

1. **Timeline plots** - Visualize experiment events chronologically
2. **Belief evolution** - Track how beliefs change over cycles
3. **Memory corruption** - Heatmaps and trends of corruption
4. **Multi-experiment comparison** - Compare metrics across experiments
5. **Network graphs** - Communication and belief alignment (multi-agent)
6. **Interactive plots** - Plotly-based interactive visualizations
7. **Export options** - Save in PNG, PDF, SVG, HTML formats

### Next Steps

- Run your own experiments
- Customize plot colors and styles
- Create custom analysis pipelines
- Build automated reporting dashboards